# Trump Twitter Data and S&P 500 Stock Prices

This notebook goes over the code needed to reproduce the results from Trump Twitter/S&P 500 Stock data analysis using WiSER, namely Table 5 of the paper.

Since this data is publically available, the data is provided in the github repo. We have attached all files needed, if you want to just go to the analysis part, please skip to [Analysis](#Analysis) Section. 

#### Packages and Reproducibility

Julia allows for easy reproducibility, by including a `Manifest.toml` and `Project.toml` pair, the user can simply run `] activate .` and the correct environment with dependencies used will run.  

In [1]:
]activate .

 Activating environment at `~/WiSER_Reproduce/trump_twitter_stock_analysis/Project.toml`


Note: We use the KNITRO solver in our analysis, which requires a KNITRO license. If you wish to run the analysis without it, you can another solver, but the results will be slightly different. Commented code is given to do this.

## Data & Availability

President Trump's tweets (excluding retweets) from November 8th, 2016 to August 31st, 2020 were used and downloaded from http://www.trumptwitterarchive.com/.

Unemployment Rate was downloaded from the Federal Reserve Economic Data (FRED) database at https://fred.stlouisfed.org/series/UNRATE. 

S&P 500 stock info was downloaded below. For the list of stocks in the S&P 500 market index, we obtained them from Wikipedia using the following python code:


```
>>> import pandas as pd
>>> table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
>>> df = table[0]
>>> df.to_csv('S&P500-Info.csv')
>>> df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
```

#### Downloading S&P 500 Data

In [2]:
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i9-9920X CPU @ 3.50GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [3]:
using Dates, MarketData, DataFrames, CSV, TimeZones, Statistics
ENV["COLUMNS"]=800
SP500stockinfo = DataFrame!(CSV.File("S&P500-Info.csv"))[!, [2, 5, 6, 7]]

# Some stocks have `stock.B` but for YahooFinance they are coded `stock-B`. Need to change to that.
SP500stockinfo[!, 1] = join.(split.(SP500stockinfo[!, 1], "."), "-");
SP500stocks = join.(split.(SP500stockinfo[!, 1], "."), "-");

Obtain daily data on each stock

In [4]:
d = DateTime(2016, 11, 6) #starting date 
df = DataFrame()
df = allowmissing(df)
tickertmp = 0 
for ticker in SP500stocks
    try
        tickertmp = allowmissing!(DataFrame!(yahoo(ticker, YahooOpt(period1=d))))
        tickertmp[!, :Stock] .= ticker
        tickertmp[!, :Yesterday_Close] = circshift(tickertmp[!, :Close], 1)
        tickertmp[1, :Yesterday_Close] = missing #this is incorrect since it brings last value to first
        append!(df, tickertmp)
    catch      # 3 have missing values 
        try
            allowmissing!(df)
            tickertmp = DataFrame!(yahoo(ticker, YahooOpt(period1=d)))
            tickertmp[!, :Stock] .= ticker
            tickertmp[!, :Yesterday_Close] = circshift(tickertmp[!, :Close], 1)
            tickertmp[1, :Yesterday_Close] = missing #this is incorrect since it brings last value to first
            append!(df, tickertmp)
        catch
            println(ticker)
        end
    end
end

In [5]:
# add variable Returns 
df[!, :Return] = (df[!, :Close] .- df[!, :Yesterday_Close]) ./ df[!, :Yesterday_Close]
df

,timestamp,Open,High,Low,Close,AdjClose,Volume,Stock,Yesterday_Close,Return
,Date?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String,Float64?,Float64?
1,2016-11-07,168.15,169.89,167.98,169.74,151.018,1.9309e6,MMM,missing,missing
2,2016-11-08,169.9,172.05,169.78,171.03,152.166,1.9557e6,MMM,169.74,0.00759982
3,2016-11-09,165.84,171.0,165.84,170.39,151.596,3.6593e6,MMM,171.03,-0.00374203
4,2016-11-10,170.16,174.67,169.79,174.28,155.057,3.28e6,MMM,170.39,0.02283
5,2016-11-11,174.05,175.13,173.73,175.08,155.769,1.8682e6,MMM,174.28,0.00459033
6,2016-11-14,175.23,175.51,171.9,172.91,153.838,2.607e6,MMM,175.08,-0.0123943
7,2016-11-15,173.16,174.25,171.91,174.24,155.021,2.1771e6,MMM,172.91,0.00769187
8,2016-11-16,173.13,173.23,171.57,172.01,154.019,2.0226e6,MMM,174.24,-0.0127985
9,2016-11-17,172.01,173.25,172.01,172.77,154.699,1.7837e6,MMM,172.01,0.0044184


In [6]:
# Join data together 
df = leftjoin(df, SP500stockinfo, on = :Stock => :Symbol)

,timestamp,Open,High,Low,Close,AdjClose,Volume,Stock,Yesterday_Close,Return,GICS Sector,GICS Sub Industry,Headquarters Location
,Date?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String,Float64?,Float64?,String?,String?,String?
1,2016-11-07,168.15,169.89,167.98,169.74,151.018,1.9309e6,MMM,missing,missing,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
2,2016-11-08,169.9,172.05,169.78,171.03,152.166,1.9557e6,MMM,169.74,0.00759982,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
3,2016-11-09,165.84,171.0,165.84,170.39,151.596,3.6593e6,MMM,171.03,-0.00374203,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
4,2016-11-10,170.16,174.67,169.79,174.28,155.057,3.28e6,MMM,170.39,0.02283,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
5,2016-11-11,174.05,175.13,173.73,175.08,155.769,1.8682e6,MMM,174.28,0.00459033,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
6,2016-11-14,175.23,175.51,171.9,172.91,153.838,2.607e6,MMM,175.08,-0.0123943,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
7,2016-11-15,173.16,174.25,171.91,174.24,155.021,2.1771e6,MMM,172.91,0.00769187,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
8,2016-11-16,173.13,173.23,171.57,172.01,154.019,2.0226e6,MMM,174.24,-0.0127985,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"
9,2016-11-17,172.01,173.25,172.01,172.77,154.699,1.7837e6,MMM,172.01,0.0044184,Industrials,Industrial Conglomerates,"St. Paul, Minnesota"


Obtain daily data macroeconomic indictaors.

In [7]:
#Treasury Yield 10 Years (^TNX)
treasuryyield10yr = DataFrame!(yahoo("^TNX", YahooOpt(period1=d)))[!, [:timestamp, :Close]]

#CBOE Volatility Index (^VIX)
volindex = DataFrame!(yahoo("^VIX", YahooOpt(period1=d)))[!, [:timestamp, :Close]]

#US Dollar Index (DX-Y.NYB)
dollarindex = DataFrame!(yahoo("DX-Y.NYB", YahooOpt(period1=d)))[!, [:timestamp, :Close]]

,timestamp,Close
,Date,Float64?
1,2016-11-07,97.78
2,2016-11-08,97.86
3,2016-11-09,98.5
4,2016-11-10,98.79
5,2016-11-13,missing
6,2016-11-14,100.11
7,2016-11-15,100.19
8,2016-11-16,100.41
9,2016-11-17,100.89


In [8]:
# Add data to stock market data as variables. Going to use close prices?
DataFrames.rename!(treasuryyield10yr, :Close => :Treasury_10YR_Yield)
DataFrames.rename!(volindex, :Close => :VIX_Vol_Index)
DataFrames.rename!(dollarindex, :Close => :Dollar_Index)

,timestamp,Dollar_Index
,Date,Float64?
1,2016-11-07,97.78
2,2016-11-08,97.86
3,2016-11-09,98.5
4,2016-11-10,98.79
5,2016-11-13,missing
6,2016-11-14,100.11
7,2016-11-15,100.19
8,2016-11-16,100.41
9,2016-11-17,100.89


In [9]:
df = leftjoin(df, treasuryyield10yr, on = :timestamp)
df = leftjoin(df, volindex, on = :timestamp)
df = leftjoin(df, dollarindex, on = :timestamp);

In [ ]:
#data save 
CSV.write("SP500dailydata.csv", df);

## Data Cleaning

This section goes over data cleaning and merging of the Twitter and Finance data.

#### Cleaning Trump Twitter Data

The data file goes until September 8th 2020, but only dates up until August 31st 2020 are used due to available US unemployment data. 

In [10]:
using Dates, DataFrames, CSV, TimeZones, Statistics
ENV["COLUMNS"]=500
tweets = DataFrame!(CSV.File("trumptweets2016_11_08_2020_09_08.csv"))

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
,String,String,String,Int64,Int64,Bool?,Int64
1,Twitter for iPhone,https://t.co/M7O6VgElFO,09-09-2020 00:32:58,912,2987,0,1303491552178614273
2,Twitter for iPhone,https://t.co/HXCqUfEuvo,09-09-2020 00:32:20,1020,3362,0,1303491394229514241
3,Twitter for iPhone,Just landed in North Carolina thank you! https://t.co/KVuTsaQ6b2,09-08-2020 22:38:50,9750,46290,0,1303462828473331712
4,Twitter for iPhone,Police Chief and most of the police in Rochester N.Y. have resigned. The Democrat Mayor and of courses Governor Cuomo have no idea what to do. New York State is a mess - No Money High Taxes &amp; Crime Everyone Fleeing. November 3rd. We can fix it!,09-08-2020 22:21:26,19720,69526,0,1303458451671011328
5,Twitter for iPhone,https://t.co/xI1wDz9qh5 https://t.co/rQ8Sl8iBwN,09-08-2020 21:53:20,5448,18698,0,1303451379596292096
6,Twitter for iPhone,Heading to North Carolina. See you soon!,09-08-2020 21:20:38,11804,83029,0,1303443150963343361
7,Twitter for iPhone,Just left Florida see everyone tonight in North Carolina! #MAGA https://t.co/pf2feOSYX7,09-08-2020 21:13:59,7066,31539,0,1303441474923749379
8,Twitter for iPhone,https://t.co/jjBLPzKSbY,09-08-2020 20:27:56,25380,91037,0,1303429887592669184
9,Twitter for iPhone,Thank you Jupiter Florida! https://t.co/B4vcal8lOF,09-08-2020 20:14:39,22196,96689,0,1303426544476581896


Times are recorded in GMT, convert times to Eastern Time in order to determine which stock day they may have had an impact on. 

In [11]:
df = DateFormat("mm-dd-yyyy HH:MM:SS");
tweets[!, :created_at] = Dates.DateTime.(tweets[!, :created_at], df);
zdt = ZonedDateTime.(tweets[!, :created_at], tz"GMT")
estDateTimes = astimezone.(zdt, tz"America/New_York")
tweets[!, :created_at] = Dates.DateTime.(estDateTimes, Local) #Here local means the time that it is set (EST)
tweets[!, :date] = Dates.Date.(estDateTimes, Local)
tweets[!, :time] = Dates.Time.(tweets[!, :created_at]);

Now to prepare tweets to match days the tweets may have had an effect on the market.

- Find days market was open -> Tweet before closing? Stay same : Move to next day
- Rest of days:
    - Bank day? Move to next day
    - Then if day is Saturday/Sunday (including adjusted bank day) -> Move to Monday
    - See if any still lie on a holiday and shift them.
    
Bank holiday dates verified and used from https://gist.github.com/shivaas/4758439.

In [17]:
bankholidays = DataFrame!(CSV.File("bankholidays.csv", header=false));
bankdates = bankholidays[!, 2];
bankday_inds = findall(map(x -> x in bankdates, tweets[!, :date]));
saturday_inds = findall(map(x -> dayname(x) in ["Saturday"], tweets[!, :date]));
sunday_inds = findall(map(x -> dayname(x) in ["Sunday"], tweets[!, :date]));
marketopeninds = setdiff(collect(1:length(tweets[!, :date])), union(bankday_inds, saturday_inds, sunday_inds));

In [18]:
# Trading hours are 9:30 a.m. to 4 p.m.
# tradestart = Dates.Time(9, 30)
tradestop = Dates.Time(16, 0)
move_inds = findall(map(x -> x > tradestop, tweets[marketopeninds, :time])); #move these to next day 

#If tweet is made after market closes, move it to the next day.
tweets[marketopeninds, :date][move_inds] = tweets[marketopeninds, :date][move_inds] .+ Dates.Day(1);

In [19]:
# Is tweet a bank holiday? Add a day 
tweets[bankday_inds, :date] = tweets[bankday_inds, :date] .+ Dates.Day(1)

# now recalculate saturday and sunday inds for bankdays that moved to saturday or sunday
saturday_inds = findall(map(x -> dayname(x) in ["Saturday"], tweets[!, :date]));
sunday_inds = findall(map(x -> dayname(x) in ["Sunday"], tweets[!, :date]));

# Is tweet on a Saturday ? Add two days 
tweets[saturday_inds, :date] = tweets[saturday_inds, :date] .+ Dates.Day(2)
# Is tweet on a Sunday ? Add one day 
tweets[sunday_inds, :date] = tweets[sunday_inds, :date] .+ Dates.Day(1)

# Now some of these days may be on a Monday that's a bank holiday. Just re-calculate if they are on a bank holiday and shift them a day again. 
bankday_inds = findall(map(x -> x in bankdates, tweets[!, :date]));
@show length(bankday_inds)
tweets[bankday_inds, :date] = tweets[bankday_inds, :date] .+ Dates.Day(1);

length(bankday_inds) = 692


In [20]:
# Concatenate tweets on the same day together 
dailytweets = combine(groupby(tweets, :date), :text => (x -> join(x, " ")) => :day_tweets)

941×2 DataFrame. Omitted printing of 1 columns
│ Row │ date       │
│     │ Date       │
├─────┼────────────┤
│ 1   │ 2020-09-08 │
│ 2   │ 2020-09-04 │
│ 3   │ 2020-09-03 │
│ 4   │ 2020-09-02 │
│ 5   │ 2020-09-01 │
│ 6   │ 2020-08-31 │
│ 7   │ 2020-08-28 │
│ 8   │ 2020-08-27 │
│ 9   │ 2020-08-26 │
│ 10  │ 2020-08-25 │
⋮
│ 931 │ 2016-11-23 │
│ 932 │ 2016-11-22 │
│ 933 │ 2016-11-21 │
│ 934 │ 2016-11-18 │
│ 935 │ 2016-11-17 │
│ 936 │ 2016-11-16 │
│ 937 │ 2016-11-15 │
│ 938 │ 2016-11-14 │
│ 939 │ 2016-11-10 │
│ 940 │ 2016-11-09 │
│ 941 │ 2016-11-08 │

In [21]:
# This reports 0, 0 indicating all tweets have been shifted 
(findall(map(x -> dayname(x) in bankdates, tweets[!, :date])), findall(map(x -> dayname(x) in ["Sunday", "Sunday"], tweets[!, :date]))) 

(Int64[], Int64[])

In [19]:
#to save data
CSV.write("dailytrumptweets_shifteddays.csv", dailytweets)

"dailytrumptweets_shifteddays.csv"

#### Merging S&P 500, Unemployment, and Trump Twitter Data

In [28]:
# load S&P data
df = DataFrame!(CSV.File("SP500dailydata.csv"))
# downloaded from https://fred.stlouisfed.org/series/UNRATE. 
unrate = DataFrame!(CSV.File("UNRATE.csv"))

# Tweets by day, linking tweets to days they could have had an effect on the market 
daily_alltrumptweets = DataFrame!(CSV.File("dailytrumptweets_shifteddays.csv"))
last(daily_alltrumptweets, 3)

,date,day_tweets
,Date,String
1,2016-11-10,Just had a very open and successful presidential election. Now professional protesters incited by the media are protesting. Very unfair! A fantastic day in D.C. Met with President Obama for first time. Really good meeting great chemistry. Melania liked Mrs. O a lot! Happy 241st birthday to the U.S. Marine Corps! Thank you for your service!! https://t.co/Lz2dhrXzo4
2,2016-11-09,Such a beautiful and important evening! The forgotten man and woman will never be forgotten again. We will all come together as never before
3,2016-11-08,"Watching the returns at 9:45pm. #ElectionNight #MAGA🇺🇸 https://t.co/HfuJeRZbod Still time to #VoteTrump! #iVoted #ElectionNight https://t.co/UZtYAY1Ba6 Don't let up keep getting out to vote - this election is FAR FROM OVER! We are doing well but there is much time left. GO FLORIDA! Just out according to @CNN: ""Utah officials report voting machine problems across entire country"" I will be watching the election results from Trump Tower in Manhattan with my family and friends. Very exciting! #ElectionDay https://t.co/MXrAxYnTjY https://t.co/FZhOncih21 We need your vote. Go to the POLLS! Let's continue this MOVEMENT! Find your poll location: https://t.co/VMUdvi1tx1… https://t.co/zGOx74Ebhw VOTE TODAY! Go to https://t.co/MXrAxYnTjY to find your polling location. We are going to Make America Great Again!… https://t.co/KPQ5EY9VwQ TODAY WE MAKE AMERICA GREAT AGAIN!"


In [29]:
# convert all tweets to lowercase
daily_alltrumptweets[!, :day_tweets] = lowercase.(daily_alltrumptweets[!, :day_tweets]);

In [30]:
# list of words to obtain daily counts for in Trump's tweets
top20_wordslist = lowercase.(["China", "Billion", "Products", "Democrats", "Great", "Dollars", "Tariffs", "Country", "Mueller", "Border", "President",
"Congressman", "People", "Korea", "Party", "Years", "Farmers", "Going", "Trade", "Never"]);

In [31]:
# function to count times a word appears in a String.
function numoccursin(word, text)
    splittext = split(text, " ")
    return length(findall(occursin.(word, splittext)))
end

#obtain the counts 
for word in top20_wordslist
    # count number of times words appeared in tweet that day 
    daily_alltrumptweets[!, Symbol(word * "_count")] = map(x -> numoccursin(word, x), daily_alltrumptweets[!, :day_tweets])
end

In [32]:
# For size considerations, don't need to keep text data
deletecols!(daily_alltrumptweets, 2)

,date,china_count,billion_count,products_count,democrats_count,great_count,dollars_count,tariffs_count,country_count,mueller_count,border_count,president_count,congressman_count,people_count,korea_count,party_count,years_count,farmers_count,going_count,trade_count,never_count
,Date,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2020-09-08,8,1,0,9,17,0,0,1,0,0,5,0,2,1,2,4,1,5,0,4
2,2020-09-04,0,0,0,0,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,2020-09-03,3,0,0,0,7,0,0,2,0,0,1,0,2,0,0,1,0,3,0,3
4,2020-09-02,0,0,0,0,4,1,0,2,0,0,1,0,3,0,3,0,0,1,0,4
5,2020-09-01,0,0,0,0,3,0,0,2,0,0,2,0,3,0,2,0,0,0,0,3
6,2020-08-31,1,0,0,2,20,0,0,3,0,0,3,0,7,0,0,1,0,6,2,7
7,2020-08-28,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8,2020-08-27,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2020-08-26,2,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0


In [33]:
# Merge S&P 500 data with Trump twitter data
df = leftjoin(df, daily_alltrumptweets, on =  :timestamp => :date);

In [34]:
# get year/month to merge the umemployment rate (monthly) data to the data set. 
df.yearmonth = Date.(Dates.Month.(df.timestamp), Dates.Year.(df.timestamp));

# merge unemployment data to dataset
df = leftjoin(df, unrate, on = :yearmonth => :DATE)

#final dataset
CSV.write("daily_trumptweets_shiftedtweetdays_snp500stocks.csv", df)

"daily_trumptweets_shiftedtweetdays_snp500stocks.csv"

<a id='Analysis'></a>
## Analysis

The following is the analysis conducted for Table 5. 

Do to filesize constraints, the final data set file `daily_trumptweets_shiftedtweetdays_snp500stocks.csv` has been gzipped and can be read in the following way. If you use an unzipped version, simply run the commented out code instead. 

In [3]:
# load packages & read in data
using DataFrames, CSV, WiSER, Statistics, StatsBase, Dates, KNITRO, CodecZlib, Mmap # last two for gzipped files. 
ENV["COLUMNS"]=1200

# tweets were shifted to match market days they could have an impact on
# shifteddf = DataFrame!(CSV.File("daily_trumptweets_shiftedtweetdays_snp500stocks.csv")) 
shifteddf = CSV.File(transcode(GzipDecompressor, Mmap.mmap("daily_trumptweets_shiftedtweetdays_snp500stocks.csv.gz"))) |> DataFrame

,timestamp,Open,High,Low,Close,AdjClose,Volume,Stock,Yesterday_Close,Return,GICS Sector,GICS Sub Industry,Headquarters Location,Treasury_10YR_Yield,VIX_Vol_Index,Dollar_Index,china_count,billion_count,products_count,democrats_count,great_count,dollars_count,tariffs_count,country_count,mueller_count,border_count,president_count,congressman_count,people_count,korea_count,party_count,years_count,farmers_count,going_count,trade_count,never_count,yearmonth,UNRATE
,Date,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String,Float64?,Float64?,String,String,String,Float64?,Float64,Float64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Date,Float64?
1,2016-11-07,168.15,169.89,167.98,169.74,151.018,1.9309e6,MMM,missing,missing,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1.828,18.71,97.78,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2016-11-01,4.7
2,2016-11-08,169.9,172.05,169.78,171.03,152.166,1.9557e6,MMM,169.74,0.00759982,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1.862,18.74,97.86,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2016-11-01,4.7
3,2016-11-09,165.84,171.0,165.84,170.39,151.596,3.6593e6,MMM,171.03,-0.00374203,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",2.072,14.38,98.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2016-11-01,4.7
4,2016-11-10,170.16,174.67,169.79,174.28,155.057,3.28e6,MMM,170.39,0.02283,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",2.117,14.74,98.79,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2016-11-01,4.7
5,2016-11-11,174.05,175.13,173.73,175.08,155.769,1.8682e6,MMM,174.28,0.00459033,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",missing,14.17,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2016-11-01,4.7
6,2016-11-14,175.23,175.51,171.9,172.91,153.838,2.607e6,MMM,175.08,-0.0123943,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",2.222,14.48,100.11,0,0,0,0,6,0,0,1,0,0,0,0,2,0,0,0,0,0,0,1,2016-11-01,4.7
7,2016-11-15,173.16,174.25,171.91,174.24,155.021,2.1771e6,MMM,172.91,0.00769187,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",2.238,13.37,100.19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-11-01,4.7
8,2016-11-16,173.13,173.23,171.57,172.01,154.019,2.0226e6,MMM,174.24,-0.0127985,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",2.223,13.72,100.41,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2016-11-01,4.7
9,2016-11-17,172.01,173.25,172.01,172.77,154.699,1.7837e6,MMM,172.01,0.0044184,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",2.276,13.35,100.89,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-11-01,4.7


Change variable names and set Financials as default sector.

In [4]:
DataFrames.rename!(shifteddf, Symbol("GICS Sector") => :Sector)
DataFrames.rename!(shifteddf, Symbol("GICS Sub Industry") => :Sub_Industry);

shifteddf[!, :Sector] = levels!(CategoricalArray(shifteddf[!, :Sector]),
    ["Financials", "Industrials", "Communication Services", "Information Technology", "Health Care", "Consumer Discretionary", "Materials", "Consumer Staples",
        "Energy", "Real Estate", "Utilities"]);

# create Return_Percentage variable (Return * 100)
shifteddf.Return_Percentage = shifteddf.Return .* 100;

Create Covid Period Variable

In [5]:
# Feb 24 2020, https://www.bbc.com/news/business-51612520 Global stock markets plunge on coronavirus fears
# In the US, the Dow Jones and S&P 500 posted their sharpest daily declines since 2018, with the Dow falling 3.5% or more than 1,000 points.

#https://www.axios.com/coronavirus-stock-market-timeline-sp-500-404ba78a-0466-4036-9506-1f981bc2689f.html 
postcovidstart = Date(Dates.Day(24), Dates.Month(2),Dates.Year(2020))

shifteddf.postcovid = shifteddf.timestamp .>= postcovidstart
# beforemarketclosedf.postcovid = beforemarketclosedf.timestamp .>= postcovidstart;

Create a lag of yesterday's returns

In [6]:
function lag(x)
    last_x = circshift(x, 1)
    last_x[1] = missing # no yesterday's return of first return 
    return last_x
end

shifteddf = transform(groupby(shifteddf, :Stock), :Return => lag => :Return_yday)
shifteddf = transform(groupby(shifteddf, :Stock), :Return_Percentage => lag => :Return_Percentage_yday);

The following constructs and fits the model. The `Optimization unsuccessful` warnings can be ignored because KNITRO by default uses a very stringent convergence criterion. FeasibleApproximate indicates the solution is adequate. Other nonlinear optimization solvers such as IPOPT will return `Optimal` status.

In [7]:
solver = KNITRO.KnitroSolver(outlev=0, ftol=2)
sp500returns_shifted_covid = WSVarLmmModel(@formula(Return_Percentage ~ 1 + Sector * postcovid + Treasury_10YR_Yield + VIX_Vol_Index + 
        Dollar_Index + UNRATE + UNRATE & postcovid + china_count + billion_count + products_count + democrats_count + great_count + 
        dollars_count + tariffs_count + country_count + mueller_count +
        border_count + president_count + congressman_count + people_count + 
        korea_count + party_count + years_count + farmers_count + going_count + trade_count + never_count + Return_Percentage_yday),
    @formula(Return_Percentage ~ 1 + Return_Percentage_yday), 
    @formula(Return_Percentage ~ 1 + Sector * postcovid + Treasury_10YR_Yield + VIX_Vol_Index + 
        Dollar_Index + UNRATE + UNRATE & postcovid + china_count + billion_count + products_count + democrats_count + great_count + 
        dollars_count + tariffs_count + country_count + mueller_count +
        border_count + president_count + congressman_count + people_count + 
        korea_count + party_count + years_count + farmers_count + going_count + trade_count + never_count), 
    :Stock, shifteddf);
@time WiSER.fit!(sp500returns_shifted_covid, solver,
    parallel = false, runs = 12, init = init_ls!(sp500returns_shifted_covid, gniters = 0)) #use knitro & LS with NO Gauss Newton iterations to initialize

### IF NO KNITRO LICENSE, comment out line 1 above and run:
# solver = Ipopt.IpoptSolver(print_level=0, watchdog_shortened_iter_trigger=3, max_iter=100)
#@time WiSER.fit!(sp500returns_shifted_covid, solver,
#    parallel = false, runs = 12, init = init_ls!(sp500returns_shifted_covid, gniters = 0)) #use knitro & LS with NO Gauss Newton iterations to initialize

┌ Warning: starting Lγ is rank deficient
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/initialization.jl:58
┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 1
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


run = 1, ‖Δβ‖ = 2.092607, ‖Δτ‖ = 0.906107, ‖ΔL‖ = 0.000000, status = FeasibleApproximate, time(s) = 8.168229
run = 2, ‖Δβ‖ = 0.056323, ‖Δτ‖ = 0.714990, ‖ΔL‖ = 0.000000, status = FeasibleApproximate, time(s) = 3.092834


┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 2
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


run = 3, ‖Δβ‖ = 0.033609, ‖Δτ‖ = 0.133543, ‖ΔL‖ = 0.000000, status = FeasibleApproximate, time(s) = 2.987264


┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 3
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


...

┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 11
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


run = 12, ‖Δβ‖ = 0.000813, ‖Δτ‖ = 0.001876, ‖ΔL‖ = 0.000004, status = FeasibleApproximate, time(s) = 5.169944
 

┌ Warning: Optimization unsuccesful; got FeasibleApproximate; run = 12
└ @ WiSER /home/cgerman/.julia/packages/WiSER/tXr2S/src/fit.jl:63


62.579199 seconds (8.31 M allocations: 410.158 MiB)



Within-subject variance estimation by robust regression (WiSER)
Number of individuals/clusters: 505
Total observations: 460330

Fixed-effects parameters:
────────────────────────────────────────────────────────────────────────────────────────────
                                                     Estimate   Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────────────────────────────────────────
β1: (Intercept)                                   0.845274     0.0743711     11.37    <1e-29
β2: Sector: Industrials                           0.0115569    0.00778216     1.49    0.1375
β3: Sector: Communication Services               -0.00910809   0.014604      -0.62    0.5328
β4: Sector: Information Technology                0.055253     0.00938808     5.89    <1e-8
β5: Sector: Health Care                           0.0241209    0.00882199     2.73    0.0063
β6: Sector: Consumer Discretionary               -0.0184413    0.00941706    -1.96    0.0502
β7: Secto

Change variable names for output.

In [8]:
mean_names = ["β$i: " for i in 1:48] .* ["Intercept", "Sector: Industrials", "Sector: Communication Services", 
    "Sector: Information Technology", "Sector: Health Care", "Sector: Consumer Discretionary", 
    "Sector: Materials", "Sector: Consumer Staples", "Sector: Energy", "Sector: Real Estate", 
    "Sector: Utilities", "COVID Period", "US Treasury 10yr Yield", "VIX Volatility Index", "US Dollar Index", 
    "US Unemployment Rate", "Tweets Count: China ", "Tweets Count: Billion", "Tweets Count: Products",
    "Tweets Count: Democrats", "Tweets Count: Great", "Tweets Count: Dollars", "Tweets Count: Tariffs", 
    "Tweets Count: Country", "Tweets Count: Mueller", "Tweets Count: Border", "Tweets Count: President", 
    "Tweets Count: Congressman", "Tweets Count: People", "Tweets Count: Korea", "Tweets Count: Party", 
    "Tweets Count: Years", "Tweets Count: Farmers", "Tweets Count: Going", 
    "Tweets Count: Trade", "Tweets Count: Never", "Previous Return Percentage", "Sector: Industrials \$\\times\$ COVID Period", 
    "Sector: Communication Services \$\\times\$ COVID Period", 
    "Sector: Information Technology \$\\times\$ COVID Period", "Sector: Health Care \$\\times\$ COVID Period", 
    "Sector: Consumer Discretionary \$\\times\$ COVID Period", 
    "Sector: Materials \$\\times\$ COVID Period", "Sector: Consumer Staples \$\\times\$ COVID Period", 
    "Sector: Energy \$\\times\$ COVID Period", "Sector: Real Estate \$\\times\$ COVID Period", 
    "Sector: Utilities \$\\times\$ COVID Period", "US Unemployment Rate \$\\times\$ COVID Period"]

wsvar_names = ["τ$i: " for i in 1:47] .* ["Intercept", "Sector: Industrials", "Sector: Communication Services", 
    "Sector: Information Technology", "Sector: Health Care", "Sector: Consumer Discretionary", 
    "Sector: Materials", "Sector: Consumer Staples", "Sector: Energy", "Sector: Real Estate", 
    "Sector: Utilities", "COVID Period", "US Treasury 10yr Yield", "VIX Volatility Index", "US Dollar Index", 
    "US Unemployment Rate", "Tweets Count: China ", "Tweets Count: Billion", "Tweets Count: Products",
    "Tweets Count: Democrats", "Tweets Count: Great", "Tweets Count: Dollars", "Tweets Count: Tariffs", 
    "Tweets Count: Country", "Tweets Count: Mueller", "Tweets Count: Border", "Tweets Count: President", 
    "Tweets Count: Congressman", "Tweets Count: People", "Tweets Count: Korea", "Tweets Count: Party", 
    "Tweets Count: Years", "Tweets Count: Farmers", "Tweets Count: Going", 
    "Tweets Count: Trade", "Tweets Count: Never", "Sector: Industrials \$\\times\$ COVID Period", 
    "Sector: Communication Services \$\\times\$ COVID Period", 
    "Sector: Information Technology \$\\times\$ COVID Period", "Sector: Health Care \$\\times\$ COVID Period", 
    "Sector: Consumer Discretionary \$\\times\$ COVID Period", 
    "Sector: Materials \$\\times\$ COVID Period", "Sector: Consumer Staples \$\\times\$ COVID Period", 
    "Sector: Energy \$\\times\$ COVID Period", "Sector: Real Estate \$\\times\$ COVID Period", 
    "Sector: Utilities \$\\times\$ COVID Period", "US Unemployment Rate \$\\times\$ COVID Period"]


sp500returns_shifted_covid.meannames .= mean_names
sp500returns_shifted_covid.wsvarnames .= wsvar_names

# results presented in Table 5.
sp500returns_shifted_covid


Within-subject variance estimation by robust regression (WiSER)
Number of individuals/clusters: 505
Total observations: 460330

Fixed-effects parameters:
──────────────────────────────────────────────────────────────────────────────────────────────────────
                                                               Estimate   Std. Error       Z  Pr(>|Z|)
──────────────────────────────────────────────────────────────────────────────────────────────────────
β1: Intercept                                               0.845274     0.0743711     11.37    <1e-29
β2: Sector: Industrials                                     0.0115569    0.00778216     1.49    0.1375
β3: Sector: Communication Services                         -0.00910809   0.014604      -0.62    0.5328
β4: Sector: Information Technology                          0.055253     0.00938808     5.89    <1e-8
β5: Sector: Health Care                                     0.0241209    0.00882199     2.73    0.0063
β6: Sector: Consumer D